Step 2. Creating a Battery Strategy

This process creates BatteryConfiguration and BatteryStrategy objects, which can be found here - http://localhost:8000/admin/simulation/.

In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from datetime import timedelta

from beo_datastore.libs.battery_schedule import create_fixed_schedule, optimize_battery_schedule

from cost.ghg.models import GHGRate
from cost.utility_rate.models import RatePlan
from der.simulation.models import BatteryConfiguration, BatterySchedule, BatteryStrategy
from load.customer.models import Meter

In [3]:
# create battery
BatteryConfiguration.objects.get_or_create(rating=150, discharge_duration_hours=4, efficiency=0.9)

(150kW @ 4 hours (90.0% efficiency), True)

In [4]:
# create a battery strategy from a RatePlan
rate_plan = RatePlan.objects.filter(name__contains="E-19").first()
frame288 = rate_plan.get_rate_frame288_by_year(2018, "energy", "weekday")

battery_strategy = BatteryStrategy.generate(
    frame288_name = "E-19 energy weekday",
    frame288=frame288,
    level=1,  # rate plans typically have only a few different rate levels per month, so this value is typically 1
    minimize=True,  # objective is to minimize bill
    discharge_threshold=0,  # only discharge down to 0kW
)

# display(frame288.dataframe)
display(battery_strategy.charge_schedule.frame288.dataframe)
display(battery_strategy.discharge_schedule.frame288.dataframe)

,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
9,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf


,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,0.0,0.0,0.0,0.0,inf,inf,inf,inf,inf,inf,0.0,0.0
9,0.0,0.0,0.0,0.0,inf,inf,inf,inf,inf,inf,0.0,0.0


In [5]:
# create a battery strategy from a GHGRate
ghg_rate = GHGRate.objects.get(name="Clean Net Short", effective__year=2030)
frame288 = ghg_rate.frame288

battery_strategy = BatteryStrategy.generate(
    frame288_name = "Clean Net Short 2030",
    frame288=frame288,
    level=8,  # Clean Net Short tables have lots of values per month, so this value should be set higher
    minimize=True,  # objective is to minimize GHG
    discharge_threshold=0,  # only discharge down to 0kW
)

# display(frame288.dataframe)
display(battery_strategy.charge_schedule.frame288.dataframe)
display(battery_strategy.discharge_schedule.frame288.dataframe)

,1,2,3,4,5,6,7,8,9,10,11,12
0,-inf,-inf,inf,inf,-inf,-inf,inf,-inf,-inf,-inf,-inf,inf
1,-inf,inf,inf,inf,inf,-inf,inf,inf,-inf,inf,inf,inf
2,inf,inf,inf,inf,inf,-inf,inf,-inf,inf,inf,inf,inf
3,inf,inf,-inf,-inf,inf,-inf,inf,inf,inf,inf,inf,inf
4,inf,inf,-inf,-inf,-inf,-inf,inf,-inf,inf,inf,inf,inf
5,inf,inf,-inf,-inf,-inf,-inf,inf,-inf,-inf,-inf,inf,inf
6,-inf,inf,-inf,inf,-inf,inf,inf,-inf,inf,-inf,-inf,inf
7,-inf,-inf,-inf,inf,inf,inf,-inf,-inf,-inf,-inf,-inf,-inf
8,inf,-inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,-inf
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,0.0,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,0.0,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,0.0,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,0.0,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,0.0,inf,0.0,inf,inf,0.0,inf,0.0,inf,inf
7,0.0,0.0,0.0,inf,inf,inf,0.0,0.0,0.0,0.0,inf,0.0
8,inf,0.0,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [6]:
# Create a Peak Demand Reduction Battery Strategy
# Charge only on NEM Exports
charge_schedule, _ = BatterySchedule.get_or_create_from_frame288(frame288=create_fixed_schedule(0, 0, 0, 0))
# Attempt to shave only the largest system peak each month (level=1), do not export to grid (threshold=0)
discharge_schedule, _ = BatterySchedule.get_or_create_from_frame288(optimize_battery_schedule(Meter.objects.first().intervalframe.maximum_frame288, level=1, charge=False, threshold=0))
battery_strategy, _ = BatteryStrategy.objects.get_or_create(name="Flatten Peak Using NEM Exports", charge_schedule=charge_schedule, discharge_schedule=discharge_schedule)

display(battery_strategy.charge_schedule.frame288.dataframe)
display(battery_strategy.discharge_schedule.frame288.dataframe)

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [7]:
# Create a Custom Battery Schedule
# Charge only on NEM Exports
charge_schedule, _ = BatterySchedule.get_or_create_from_frame288(frame288=create_fixed_schedule(0, 0, 0, 0))
# Discharge from 4 p.m. to 9:59 p.m. without exporting to the grid
discharge_schedule, _ = BatterySchedule.get_or_create_from_frame288(frame288=create_fixed_schedule(16, 22, 0, float("inf")))

battery_strategy, _ = BatteryStrategy.objects.get_or_create(name="Flatten Evening Load Using NEM Exports", charge_schedule=charge_schedule, discharge_schedule=discharge_schedule)

display(battery_strategy.charge_schedule.frame288.dataframe)
display(battery_strategy.discharge_schedule.frame288.dataframe)

,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
9,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
